In [1]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.7 MB 5.4 MB/s eta 0:00:03
   ------ --------------------------------- 2.1/13.7 MB 5.4 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.7 MB 5.3 MB/s eta 0:00:02
   ------------ --------------------------- 4.2/13.7 MB 5.2 MB/s eta 0:00:02
   --------------- ------------------------ 5.2/13.7 MB 5.1 MB/s eta 0:00:02
   ------------------ --------------------- 6.3/13.7 MB 5.1 MB/s eta 0:00:02
   --------------------- ------------------ 7.3/13.7 MB 5.1 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/13.7 MB 5.1 MB/s eta 0:00:02
   --------------------------- ------------ 9.4/13.7 MB 5.1 MB/s eta 0:00:01
   ------------------------------ --------- 10.5/13.7 MB 5.1 MB/s eta 0:00:01
   ---------------------------------- ----- 11.8/13.7 MB 5.1 MB/s eta 0:00:01
   ------------------------------------ --- 12.3/13.7 MB 5.1 MB/s eta 0:00:01
   


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

def extract_doc_id(url):
    import re
    match = re.search(r'/d/([a-zA-Z0-9-_]+)', url)
    return match.group(1) if match else None

def return_TableContent(table):
    z=0
    coordinates = { 0: "" }
    for row in table.get("tableRows"):
        i = 0
        element = {
                0:"", 
                1:"", 
                2:""}                    
        for cell in row.get("tableCells"):                          
            for content in cell.get("content"):
                paragraph = content.get("paragraph")
                for run in paragraph.get("elements"):                                
                    text_run = run.get("textRun")
                    if text_run:
                        contentCell = text_run.get("content").strip()
                        if z != 0 and i < 3:  # Assuming we want to limit to 3 columns
                            element[i] = contentCell
                            i += 1
        if i > 0:
            coordinates[z] = element 
        z += 1                  
    coordinates.pop(0, None)  # Remove the initial empty entry

    return coordinates     

def print_UnicodeCaracter(x, y, char):
    """Prints a character at the specified (x, y) coordinates."""
    # ANSI escape code to move cursor: \033[<Y>;<X>H
    # Y is row, X is column. Terminals are typically 1-indexed.
    # So, add 1 to x and y for common 0-indexed programming coordinates.
    print(f"\033[{y + 1};{x + 1}H{char}", end="")     


def read_google_doc(doc_url):
    # Extract the document ID from the URL
    doc_id = extract_doc_id(doc_url)
    
    # If modifying these scopes, delete the file token.json.
    SCOPES = ["https://www.googleapis.com/auth/documents.readonly"]
    
    if not doc_id:
        print("Invalid Google Doc URL.")
        return None
    
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("docs", "v1", credentials=creds)

        # Retrieve the documents contents from the Docs service.
        document = service.documents().get(documentId=doc_id).execute()
        # Print the content of the document
        for element in document.get("body").get("content"):
            if "table" in element:
                
                table = element.get("table")
                coordinatesDict = return_TableContent(table)
                qtdCoordinates = len(coordinatesDict)
                print(f"Number of coordinates: {qtdCoordinates}")
                while qtdCoordinates > 0:
                    #if qtdCoordinates in coordinatesDict:
                    element = coordinatesDict[qtdCoordinates]
                    print(f"Element: {element}")
                    qtdCoordinates -= 1
                # print(f"Table content: {coordinatesDict}")
                # for coord in coordinatesDict:
                #     element = coordinatesDict[coord]
                #     print_UnicodeCaracter(int(element[0]), int(element[2]), element[1])                
                
    except HttpError as err:
        print(err)

read_google_doc("https://docs.google.com/document/d/1M0QY4-iKNHwPI3nS5LZBYDWXIpQzqypyzS_doHVHUEU/edit?tab=t.0")

ValueError: Client secrets must be for a web or installed app.